In [1]:
import json
from functools import lru_cache
from operator import itemgetter
from typing import List
import boto3

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [2]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [3]:
session = boto3.Session(profile_name='master')

In [4]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [5]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int

In [6]:
# list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [7]:
import awswrangler as wr


In [8]:
s3 = session.client('s3')

In [9]:
def get_json_from_s3(bucket, key, session):
    s3 = session.client('s3')
    obj = s3.get_object(Bucket=bucket, Key=key)
    data = obj['Body'].read().decode('utf-8')
    return json.loads(data)

In [10]:
res_empty = get_json_from_s3("kosmobiker-masterproject", 'data/raw/ape_in_records_historical_prices.json', session)
res_bit = get_json_from_s3("kosmobiker-masterproject", 'data/raw/bitcoin_historical_prices.json', session)
res_amar = get_json_from_s3("kosmobiker-masterproject", 'data/raw/amateras_historical_prices.json', session)
res_aaa = get_json_from_s3("kosmobiker-masterproject", 'data/raw/american-shiba_historical_prices.json', session)

In [11]:
len(res_empty['prices'])

1

In [12]:
def make_df(json_data):
    return pd.DataFrame({
                "timestamp" : [i[0] for i in json_data['prices']],
                "prices" : [i[1] for i in json_data['prices']],
                "market_caps" : [i[1] for i in json_data['market_caps']],
                "total_volumes" : [i[1] for i in json_data['total_volumes']]
    })
    


In [13]:
import jsonschema
def validate_json(data, schema):
    try:
        validate(instance=data, schema=schema)
        if len(data['prices']) < 30:
            raise ValueError
        return True
    except Exception:
        return False



In [38]:
s3 = session.client('s3')

def ListFilesV1(client, bucket, prefix=''):
    """List files in specific S3 URL"""
    res = []
    paginator = client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket,
                                    Prefix=prefix)
    for page in page_iterator:
        for content in page.get('Contents'):
            yield content.get('Key')
            # res.append(content.get('Key'))
    # return res
gen = ListFilesV1(s3, 'kosmobiker-masterproject', "data/raw")

In [4]:
import boto3


def get_matching_s3_objects(bucket, prefix="", suffix=""):
    """
    Generate objects in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch objects whose key starts with
        this prefix (optional).
    :param suffix: Only fetch objects whose keys end with
        this suffix (optional).
    """
    session = boto3.Session(profile_name='master')
    s3 = session.client("s3")
    paginator = s3.get_paginator("list_objects_v2")

    kwargs = {'Bucket': bucket}

    # We can pass the prefix directly to the S3 API.  If the user has passed
    # a tuple or list of prefixes, we go through them one by one.
    if isinstance(prefix, str):
        prefixes = (prefix, )
    else:
        prefixes = prefix

    for key_prefix in prefixes:
        kwargs["Prefix"] = key_prefix

        for page in paginator.paginate(**kwargs):
            try:
                contents = page["Contents"]
            except KeyError:
                break

            for obj in contents:
                key = obj["Key"]
                if key.endswith(suffix):
                    yield obj


def get_matching_s3_keys(bucket, prefix="", suffix=""):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    :param suffix: Only fetch keys that end with this suffix (optional).
    """
    for obj in get_matching_s3_objects(bucket, prefix, suffix):
        yield obj["Key"]


In [5]:
gen = get_matching_s3_keys('kosmobiker-masterproject', suffix=".json")

In [17]:
def list_test(): 
    session = boto3.Session(profile_name='master')
    client = session.client('s3')
    paginator = client.get_paginator('list_objects_v2')
    operation_parameters = {'Bucket': 'kosmobiker-masterproject',
                            'Prefix': 'data/raw'}
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        response = page['Contents']
        for obj in response:
            yield obj['Key']


In [18]:
gen = list_test()

In [20]:
for _ in gen:
    print(_)

data/raw/0-5x-long-algorand-token_historical_prices.json
data/raw/0-5x-long-altcoin-index-token_historical_prices.json
data/raw/0-5x-long-ascendex-token-token_historical_prices.json
data/raw/0-5x-long-bitcoin-cash-token_historical_prices.json
data/raw/0-5x-long-bitcoin-token_historical_prices.json
data/raw/0-5x-long-cardano-token_historical_prices.json
data/raw/0-5x-long-chainlink-token_historical_prices.json
data/raw/0-5x-long-cosmos-token_historical_prices.json
data/raw/0-5x-long-dogecoin-token_historical_prices.json
data/raw/0-5x-long-eos-token_historical_prices.json
data/raw/0-5x-long-ethereum-classic-token_historical_prices.json
data/raw/0-5x-long-ethereum-token_historical_prices.json
data/raw/0-5x-long-kyber-network-token_historical_prices.json
data/raw/0-5x-long-matic-token_historical_prices.json
data/raw/0-5x-long-privacy-index-token_historical_prices.json
data/raw/0-5x-long-swipe-token_historical_prices.json
data/raw/0-5x-long-theta-network-token_historical_prices.json
data/ra